In [1]:
import numpy as np
def reweight_distribution(orginal_distribution, temperature=0.5):
    distribution=np.log(orginal_distribution)/temperature
    distribution=np.exp(distribution)
    return distribution/np.sum(distribution)

In [2]:
import keras

path=keras.utils.get_file('nietzsche.txt',origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text=open(path).read().lower()
print('Corpus leght: ', len(text))

Corpus leght:  600901


In [3]:
# wyodrebnienie sekwencji składającej się z 60 znaków
maxlen=60

step=3

sentences=[]
next_chars=[]
for i in range(0,len(text)-maxlen,step):
    sentences.append(text[i:i+maxlen])
    next_chars.append(text[i+maxlen])

print('Liczba sekwencji: ',len(sentences))

chars=sorted(list(set(text)))
print('liczba unikatowych znaków: ', len(chars))
char_indices=dict((char, chars.index(char))for char in chars)

x=np.zeros((len(sentences),maxlen,len(chars)),dtype=np.bool)
y=np.zeros((len(sentences),len(chars)),dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):

        x[i,t,char_indices[char]]=1
    y[i,char_indices[next_chars[i]]]=1


Liczba sekwencji:  200281
liczba unikatowych znaków:  59


C:\Users\abuka\AppData\Local\Temp\ipykernel_9736\300909591.py:18: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  x=np.zeros((len(sentences),maxlen,len(chars)),dtype=np.bool)
C:\Users\abuka\AppData\Local\Temp\ipykernel_9736\300909591.py:19: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y=np.zeros((len(sentences),len(chars)),dtype=np.bool)


In [4]:
#LSTM
from keras import layers
model=keras.models.Sequential()
model.add(layers.LSTM(128,input_shape=(maxlen,len(chars))))
model.add(layers.Dense(len(chars),activation='softmax'))

optimizer=keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy',optimizer=optimizer)


C:\Users\abuka\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\optimizers\legacy\rmsprop.py:143: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [5]:
def sample(preds,temperature=1.0):
    preds=np.asarray(preds).astype('float64')
    preds=np.log(preds)/temperature
    exp_preds=np.exp(preds)
    preds=exp_preds/np.sum(exp_preds)
    probas=np.random.multinomial(1,preds,1)
    return np.argmax(probas)

In [7]:
#pentla genrująca tekst
import random
import sys

for epoch in range (1,60):
    print('epoka',epoch)
    model.fit(x,y,batch_size=128,epochs=1)
    start_index=random.randint(0,len(text)-maxlen-1)
    generated_text=text[start_index:start_index+maxlen]
    print('Generowanie przy użyciu tekstu początkowego: '+generated_text)

    for temperature in [0.2,0.5,1.0,1.2]:
        print('Wartość parametru:' , temperature)
        sys.stdout.write(generated_text)
        for i in range(400):
            sampled=np.zeros((1,maxlen,len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0,t,char_indices[char]]=1

            preds=model.predict(sampled,verbose=0)[0]
            next_index=sample(preds,temperature)
            next_char=chars[next_index]
            generated_text+=next_char
            generated_text=generated_text[1:]

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

epoka 1
1565/1565 [==============================] - 132s 84ms/step - loss: 1.6210
Generowanie przy użyciu tekstu początkowego: eated by men like birds, which, losing
their way, have come 
Wartość parametru: 0.2
eated by men like birds, which, losing
their way, have come and to the sense as a something the destines to the sense as the sense and as the sentiment and the person of the assively to the stander to the present the persons to the sense as a sense to the present to the present to the person and all the sense to see to the present to the sense to the and perhaps to the are to the present to the sense to the free to the present to the failing the present to
Wartość parametru: 0.5
nse to the free to the present to the failing the present to refersed it be to be the philosophers from the speciate present contine to least the failing to the will proces on e his fact and need to periously many the are to an extenserance and even to in the capays in first to the all the assessed to a

C:\Users\abuka\AppData\Local\Temp\ipykernel_9736\4029115235.py:3: RuntimeWarning: divide by zero encountered in log
  preds=np.log(preds)/temperature


we worth and concesls.

181. the welfare for our responsibility, the spirit and contemplritt. ye hus ekating out of attempten courage any lights: a consciences upon the people to maca--scentsige that wishruten, is not without a stronged privaly ninger to neld with "weer anmans of the stand" in
many and power, of the demanded, which eleved, somethint of infecioses.=--that he did now
Wartość parametru: 1.2
ed, which eleved, somethint of infecioses.=--that he did now
exterdance.. bide), it be a ttering of out of trainousch "iwn sought
as the pare
explanation itself, ariset; but to dos the spridicies. even to adpro reductped etakee" is all intentirence,
as all trains, which
are thus atuning in the moralestly aristo slavaly seems, as thoscerede and
pignedming and for withiain good intestiname-geen the complesoralice recognishment day
men to cexedicance and name
epoka 36
1565/1565 [==============================] - 123s 79ms/step - loss: 1.2769
Generowanie przy użyciu tekstu początkowego: an